In [61]:
import rdflib
from rdflib import FOAF, RDF, URIRef, Literal, SKOS, Namespace
import datetime as dt

# Creating example patient demographics data

In [62]:
# I'm making up a new namespace to define the property 'id'

CUSTOM = Namespace('http://example.org/customNamespace/')

# I'm using the schema birthdate property
SCHEMA = Namespace('https://schema.org/')

Alice = URIRef('http://example.org/people/Alice')
Heidi = URIRef('http://example.org/people/Heidi')

g = rdflib.Graph()

g.add((Alice, CUSTOM.id, Literal('12345')))
g.add((Alice, RDF.type, FOAF.Person))
g.add((Alice, FOAF.name, Literal('Alice Williams')))
g.add((Alice, SCHEMA.birthDate, Literal(dt.date(1981, 3,4))))


g.add((Heidi, CUSTOM.id, Literal('5678')))
g.add((Heidi, RDF.type, FOAF.Person))
g.add((Heidi, FOAF.name, Literal('Heidi Cook')))
g.add((Heidi, SCHEMA.birthDate, Literal(dt.date(1991, 3,10))))

# Querying the patients
For this quick prototype we're assuming that the URI of the elements serve as patient ids.
The fieldnames in the `SELECT` statement should match the fieldnames required in the fhir resources.

In [63]:
query = '''
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX custom: <http://example.org/customNamespace/>
PREFIX schema: <https://schema.org/>

SELECT ?id ?name ?birthDate
WHERE{

?person a foaf:Person ;
    foaf:name ?name ;
    custom:id ?id ;
    schema:birthDate ?birthDate
}
'''

In [64]:
result = g.query(query)

for r in g.query(query):
    for k, v in r.asdict().items():
        print(f'{k}: {v}')
        
    print('')

id: 12345
name: Alice Williams
birthDate: 1981-03-04

id: 5678
name: Heidi Cook
birthDate: 1991-03-10



# Converting triples to fhir resources
The properties we have extracted from the triplestore are close to the structure required by the fhir resources. However, some properties in the fhir resources have a more nested structure than our list of dicts currently has.
For example, the Patient resource contains a field *name* which should contain a *list* of items of the type *HumanName*. The type *HumanName* is a fhir type that contains multiple fields such as *given* and *family*.

The `create_patient` function maps the flat dicts into a more hierachical structure that can be parsed by the `Patient` class in `fhir.resources`.

In [65]:
from fhir.resources.patient import Patient
from fhir.resources.address import Address
from fhir.resources.fhirtypes import Date
from fhir.resources.humanname import HumanName

In [66]:
def create_patient(properties):
    # Some items in the patient resource require some extra work
    properties = dict(properties)
    properties['name'] = [HumanName(given=[properties['name']])]
    
    birthdate= properties['birthDate'].toPython()
    properties['birthDate'] = Date(birthdate.year, birthdate.month, birthdate.day)
    
    return Patient(**properties)

In [67]:
def create_patients():
    for r in result:
        d = r.asdict()
        patient = create_patient(d)
        yield patient
        
        
patients = list(create_patients())

print('Patients as fhir resources:')
for p in patients:
    print(p.json())

Patients as fhir resources:
{"id": "12345", "birthDate": "1981-03-04", "name": [{"given": ["Alice Williams"]}], "resourceType": "Patient"}
{"id": "5678", "birthDate": "1991-03-10", "name": [{"given": ["Heidi Cook"]}], "resourceType": "Patient"}
